In [36]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import numpy as np
from collections import namedtuple

In [19]:
url = 'https://ottoneu.fangraphs.com/953/transactions'
r = requests.get(url)
c = r.content

In [20]:
soup = BeautifulSoup(c, 'html.parser')

In [34]:
table_name = soup.find('main').find('h1').get_text()

In [37]:
table = soup.find('main').find('table')

In [56]:
rows = table.find_all('tr')

data = []
for idx, row in enumerate(rows):
    if idx == 0:
        cols = row.find_all('th')
    else:
        cols = row.find_all('td')
    cols = [ele.text.strip() for ele in cols]
    data.append([ele for ele in cols])

In [59]:
transactions = pd.DataFrame(data[1:], columns=data[0])

https://ottoneu.fangraphs.com/953/transactions?page=34 (starts at zero index)

might need to just walk through every page either with selenium or indexing

set it to scrape every sunday. save existing transactions to csv, add new ones, and drop duplicates
   - only scrape first page. if no duplicates, then scrape next page

In [63]:
transactions.head()

,Date,Transaction Type,Player Name,Team Name,From Team,Salary
0,"Apr 21, 2019 11:46:58",cut,Nick Pivetta,Bobblehead,,$2
1,"Apr 20, 2019 23:10:32",cut,Jon Lester,St. Clair Gorillas,,$4
2,"Apr 20, 2019 16:49:01",add,Hector Neris,Clueless Joes,,$5
3,"Apr 20, 2019 08:00:01",add,Sam Gaviglio,Clueless Joes,,$2
4,"Apr 19, 2019 14:02:02",add,Jason Heyward,Clueless Joes,,$8


In [65]:
transactions['Team Name'].value_counts()

Bobblehead            10
Enders Game            9
Urbina Blight          9
VORP Speed             5
St. Clair Gorillas     4
Clueless Joes          4
the balking dead       3
Peons                  3
The Hobo Jungle        2
Scotty Smalls          1
Name: Team Name, dtype: int64

In [95]:
# probably need Selenium to comb through the auction results
auction_results_url = 'https://ottoneu.fangraphs.com/953/auctionresults?id=384614'
ar = requests.get(auction_results_url)
ar_c = ar.content

ar_soup = BeautifulSoup(ar_c, 'html.parser')

In [96]:
ar_head = ar_soup.find('main').find('h2').get_text().split()

In [97]:
ar_head.split()

['Auction', 'for', 'Sam', 'Gaviglio', 'TOR', 'SP/RP', 'R']

In [106]:
player_name = [x for x in ar_head[1:-1] if x.istitle()]
player_team = ar_head[-3]
player_pos = ar_head[-2]
player_hand = ar_head[-1]

In [99]:
started_by = ' '.join(ar_soup.find_all('h3')[1].get_text().split()[3:])

In [100]:
started_by

'St. Clair Gorillas'

In [101]:
all_bids = ar_soup.find('table')

In [102]:
bid_rows = all_bids.find_all('tr')

bid_data = []
for idx, row in enumerate(bid_rows):
    if idx == 0:
        cols = row.find_all('th')
    else:
        cols = row.find_all('td')
    cols = [ele.text.strip() for ele in cols]
    bid_data.append([ele for ele in cols])

In [103]:
bid_data

[['Team', 'Bid'], ['Clueless Joes', '$5'], ['St. Clair Gorillas', '$1']]